# 2. Develop your segmentation model

In [35]:
#Import libraries
import os
from model_class import ModelSegmentation
from aux_functions import slicing
import cv2
#Inputs
working_directory="C:/Users/Pheno/Documents/database_almondcv2/"
pictures_directory=os.path.join(working_directory, "fotos_prueba_seed_2022")
pre_model=os.path.join(working_directory, "models/yolo11s-seg.pt")
model_path=os.path.join(working_directory, "models/seed_2022_yolov11s_320.pt")
output_directory=os.path.join(working_directory,"output_directory")

### Slicing

In [ ]:
#Slice_pictures for training
slicing(input_folder=pictures_directory,output_directory=working_directory,name_slicing="Slices_probando", number_pictures=4, slice_height=320, slice_width=320)

### Training

In [ ]:
#Label with CVAT

zip_file_shell=os.path.join(working_directory,"")

In [ ]:
#Model segmentation training

model=ModelSegmentation(working_directory=working_directory)
model.train_segmentation_model(input_zip=zip_file_shell, epochs=100,imgsz=640, name_segmentation="shell_2022_yolov11",
                                      pre_model=pre_model, batch=16)

### Deploy and reconstruct a picture

#### Slice_predict_reconstruct approach

In [37]:
# Join patches approach

model=ModelSegmentation(working_directory=working_directory)
masks=model.slice_predict_reconstruct(input_folder=pictures_directory,imgsz=320, model_path=model_path,
                                          slice_height=320, slice_width=320,overlap_height_ratio=0.2,
                                          overlap_width_ratio=0.2)

01/17/2025 17:34:49 - INFO - sahi.slicing -   image.shape: (756, 2934)
01/17/2025 17:34:49 - INFO - sahi.slicing -   Num slices: 36 slice_height: 320 slice_width: 320


GPU detectada: NVIDIA GeForce RTX 3060
Memoria total de la GPU: 12.00 GB
Image1/2


01/17/2025 17:34:56 - INFO - sahi.slicing -   image.shape: (756, 2951)
01/17/2025 17:34:56 - INFO - sahi.slicing -   Num slices: 36 slice_height: 320 slice_width: 320


Image2/2


In [44]:
# To show the masks

for mask in masks:
    cv2.imwrite(f"{output_directory}/{os.path.basename(mask[1])}", mask[0])


#### SAHI

In [45]:
model=ModelSegmentation(working_directory=working_directory)
masks=model.predict_model_sahi(model_path=model_path, check_result=False, folder_input=pictures_directory,
                                            retina_masks=True,
                                              postprocess_match_threshold=0.2, overlap_height_ratio=0.2,
                                                overlap_width_ratio=0.2, postprocess_match_metric="IOS", 
                                                postprocess_type="GREEDYNMM", slice_height=320, slice_width=320,
                                                  confidence_treshold=0.95,
                                                  imgsz=320)

GPU detectada: NVIDIA GeForce RTX 3060
Memoria total de la GPU: 12.00 GB
Pic 1/2
Performing prediction on 36 slices.
Pic 2/2
Performing prediction on 36 slices.


In [50]:
# To show the masks
for mask in masks:
    mask[0].export_visuals(export_dir=output_directory, hide_labels=True, rect_th=1, file_name=f"{os.path.basename(mask[1])}")